# Inferential Analysis with Multinomial Logistic Regression

### Imports

In [1]:
options(repr.matrix.max.rows = 6)
library(VGAM)
library(cowplot)
library(tidyverse)
library(broom)
library(scales)
library(foreign)
library(MASS)
library(dplyr)

Loading required package: stats4

Loading required package: splines

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0     ✔ purrr   1.0.1
✔ tibble  3.1.8     ✔ dplyr   1.1.0
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘scales’


The following object is masked from ‘package:purrr’:

    discard


The following object is masked from ‘package:readr’:

    col_factor



Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select




In [2]:
# function to see tidy outputs for GLM

tidy.vglm <- function(x, conf.int=FALSE, conf.level=0.95) {
    co <- as.data.frame(coef(summary(x)))
    names(co) <- c("estimate","std.error","statistic","p.value")
    if (conf.int) {
        qq <- qnorm((1+conf.level)/2)
        co <- transform(co,
                        conf.low=estimate-qq*std.error,
                        conf.high=estimate+qq*std.error)
    }
    co <- data.frame(term=rownames(co),co)
    rownames(co) <- NULL
    return(co)
}

### Loading Data

In [3]:
# Load the data
df <- suppressMessages(read_csv("../data/processed/data.csv"))
df <- df[,-1]
head(df)


age,education,spouse_education,children,religion,work,spouse_occupation,living_standard,media_exposure,contraceptive_method
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
24,2,3,3,1,1,2,3,0,1
45,1,3,10,1,1,3,4,0,1
43,2,3,7,1,1,3,4,0,1
42,3,2,9,1,1,3,3,0,1
36,3,3,8,1,1,3,2,0,1
19,4,4,0,1,1,3,3,0,1


### Data Wrangling

In [4]:
# encode variables properly

df$contraceptive_method <- as.factor(df$contraceptive_method)
df$education <- as.factor(df$education)
df$living_standard <- as.factor(df$living_standard)
df$work <- as.factor(df$work)
df$media_exposure <- as.factor(df$media_exposure)
df$religion <- as.factor(df$religion)

head(df)

age,education,spouse_education,children,religion,work,spouse_occupation,living_standard,media_exposure,contraceptive_method
<dbl>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<fct>,<fct>,<fct>
24,2,3,3,1,1,2,3,0,1
45,1,3,10,1,1,3,4,0,1
43,2,3,7,1,1,3,4,0,1
42,3,2,9,1,1,3,3,0,1
36,3,3,8,1,1,3,2,0,1
19,4,4,0,1,1,3,3,0,1


In [5]:
# check response levels
levels(df$contraceptive_method)

[1] "1" "2" "3"

In [6]:
df$contraceptive_method <- fct_relevel(df$contraceptive_method, c('3', '2', '1'))
levels(df$contraceptive_method)

[1] "3" "2" "1"

This means that `1` (no contraception) will be the reference level for our multinomial regression model. According to the data description, we have 1=No-use, 2=Long-term, 3=Short-term

In [7]:
# encode response to make more comprehendable
# df <- df |>
#     mutate(contraceptive_method = case_when(
#         contraceptive_method == "1" ~ "No Use",
#         contraceptive_method == "2" ~ "Long term",
#         contraceptive_method == "3" ~ "Short term"
# ))
# head(df)
# tail(df)

In [8]:
# check repsonse levels now
# df$contraceptive_method <- as.factor(df$contraceptive_method)
# levels(df$contraceptive_method)

### Multinomial Logistic model - Simple regressions

### `age`

In [9]:
# define the model
multinomial_model1 <- vglm(contraceptive_method ~ age, 
                           family = multinomial, 
                           data = df)

In [10]:
summary_multinomial_model1 <- tidy.vglm(multinomial_model1,
  conf.int = TRUE, conf.level = 0.95
) 
summary_multinomial_model1

term,estimate,std.error,statistic,p.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept):1,1.37442197,0.249142813,5.516603,3.456151e-08,0.886111032,1.86273291
(Intercept):2,-1.11451208,0.288155017,-3.867752,1.098434e-04,-1.679285534,-0.54973862
age:1,-0.04975766,0.007654227,-6.500678,7.995903e-11,-0.064759673,-0.03475565
age:2,0.01411422,0.008224673,1.716083,8.614693e-02,-0.002005845,0.03023428


This shows that `age` is significantly associated with long term methods used at statistical significance level of 0.05 with $p<.001$. However, we do not have enough statistical evidence to associate `age` with short term contraceptive methods.

In [11]:
tibble(summary_multinomial_model1[3, 1:2], exp.estimate = round(exp(summary_multinomial_model1[3, 2]), 2))

term,estimate,exp.estimate
<chr>,<dbl>,<dbl>
age:1,-0.04975766,0.95


This means that with each year's increase in age, the woman is $\frac{1}{0.95} = 1.05$ times more likely to use no contraceptive methods as opposed to long term contraception.

#### `children`

In [12]:
# define the model
multinomial_model2 <- vglm(contraceptive_method ~ children, 
                           family = multinomial, 
                           data = df)

In [13]:
summary_multinomial_model2 <- tidy.vglm(multinomial_model2,
  conf.int = TRUE, conf.level = 0.95
) 
summary_multinomial_model2

term,estimate,std.error,statistic,p.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept):1,-0.46259001,0.10229255,-4.522226,6.119277e-06,-0.66307972,-0.2621003
(Intercept):2,-1.11316315,0.11916411,-9.341430,9.504205e-21,-1.34672051,-0.8796058
children:1,0.08124393,0.02639325,3.078209,2.082487e-03,0.02951412,0.1329737
children:2,0.14404812,0.02872997,5.013863,5.334794e-07,0.08773842,0.2003578


This means that the number of children is statistically associated with both long term and shot term contraceptive methods at a significance level of 0.05. 

In [14]:
tibble(summary_multinomial_model2[3:4, 1:2], exp.estimate = round(exp(summary_multinomial_model2[3:4, 2]), 2))


term,estimate,exp.estimate
<chr>,<dbl>,<dbl>
children:1,0.08124393,1.08
children:2,0.14404812,1.15


This means that with a unit increase in the number of children, a woman is 1.08 times more likely to use long term contraception than no contraception and 1.15 times more likely to use short term contraception as opposed to no contraception

#### `religion`

In [15]:
multinomial_model3 <- vglm(contraceptive_method ~ religion, 
                           family = multinomial, 
                           data = df)

In [16]:
summary_multinomial_model3 <- tidy.vglm(multinomial_model3,
  conf.int = TRUE, conf.level = 0.95
) 
summary_multinomial_model3

term,estimate,std.error,statistic,p.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept):1,-0.08338161,0.1668115,-0.49985519,6.171770e-01,-0.4103262,0.2435630
(Intercept):2,0.01324523,0.1627613,0.08137825,9.351411e-01,-0.3057610,0.3322514
religion1:1,-0.14247320,0.1785878,-0.79777697,4.249999e-01,-0.4924988,0.2075524
religion1:2,-0.78133383,0.1794083,-4.35506000,1.330305e-05,-1.1329676,-0.4297001


This shows statistical significance of only short term contraception with religion.

In [17]:
tibble(summary_multinomial_model3[3, 1:2], exp.estimate = round(exp(summary_multinomial_model3[3, 2]), 2))


term,estimate,exp.estimate
<chr>,<dbl>,<dbl>
religion1:1,-0.1424732,0.87


This means that when the woman is Muslim, she is $\frac{1}{0.87} = 1.15$ times more likely to use no contraception rather than short term contraception.

#### `education`

In [18]:
multinomial_model4 <- vglm(contraceptive_method ~ education, 
                           family = multinomial, 
                           data = df)

In [19]:
summary_multinomial_model4 <- tidy.vglm(multinomial_model4,
  conf.int = TRUE, conf.level = 0.95
) 
summary_multinomial_model4[3:8,]

,term,estimate,std.error,statistic,p.value,conf.low,conf.high
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,education2:1,0.5711561,0.2205790,2.589350,9.615734e-03,0.1388293,1.003483
4,education2:2,0.8779383,0.3918238,2.240646,2.504903e-02,0.1099778,1.645899
5,education3:1,0.8244887,0.2165056,3.808163,1.400032e-04,0.4001454,1.248832
6,education3:2,1.6547451,0.3728691,4.437872,9.085274e-06,0.9239351,2.385555
7,education4:1,1.0540631,0.2134275,4.938742,7.862831e-07,0.6357530,1.472373
8,education4:2,2.6054372,0.3624393,7.188616,6.545113e-13,1.8950692,3.315805


This shows statistically significant association between all levels of education and uuse of both short and long term contraception.

In [20]:
tibble(summary_multinomial_model4[3:8, 1:2], exp.estimate = round(exp(summary_multinomial_model4[3:8, 2]), 2))


term,estimate,exp.estimate
<chr>,<dbl>,<dbl>
education2:1,0.5711561,1.77
education2:2,0.8779383,2.41
education3:1,0.8244887,2.28
education3:2,1.6547451,5.23
education4:1,1.0540631,2.87
education4:2,2.6054372,13.54


- A woman with medium-low (`2`) level of education is 1.77 times more likely than a woman with low(`1`) level of education to use long term contraception and 2.41 times more likely to use short term contraception as opposed to no contraception.

- A woman with medium-high (`3`) level of education is 2.28 times more likely than a woman with low(`1`) level of education to use long term contraception and 5.23 times more likely to use short term contraception as opposed to no contraception.

- A woman with high (`4`) level of education is 2.87 times more likely than a woman with low(`1`) level of education to use long term contraception and 13.54 times more likely to use short term contraception as opposed to no contraception.

#### `work`

In [21]:
multinomial_model5 <- vglm(contraceptive_method ~ work, 
                           family = multinomial, 
                           data = df)

In [22]:
summary_multinomial_model5 <- tidy.vglm(multinomial_model5,
  conf.int = TRUE, conf.level = 0.95
) 
summary_multinomial_model5

term,estimate,std.error,statistic,p.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept):1,-0.43531807,0.1223653,-3.55752930,3.743593e-04,-0.67514961,-0.1954865
(Intercept):2,-0.64716207,0.1308370,-4.94632484,7.562772e-07,-0.90359778,-0.3907264
work1:1,0.30022929,0.1401630,2.14200045,3.219345e-02,0.02551478,0.5749438
work1:2,0.01528008,0.1529553,0.09989902,9.204245e-01,-0.28450676,0.3150669


In [23]:
tibble(summary_multinomial_model5[3, 1:2], exp.estimate = round(exp(summary_multinomial_model5[3, 2]), 2))


term,estimate,exp.estimate
<chr>,<dbl>,<dbl>
work1:1,0.3002293,1.35


There is statistical association between work and long term contraception. The association can be quantified as: A woman who works is 1.35 times more likely than the one who doesn't to use long term contraception as opposed to no contraception.

#### `living_standard`

In [24]:
multinomial_model6 <- vglm(contraceptive_method ~ living_standard, 
                           family = multinomial, 
                           data = df)

In [30]:
summary_multinomial_model6 <- tidy.vglm(multinomial_model6,
  conf.int = TRUE, conf.level = 0.95
) 
summary_multinomial_model6

term,estimate,std.error,statistic,p.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept):1,-0.6931472,0.1936493,-3.579395,3.443906e-04,-1.0726928,-0.3136016
(Intercept):2,-2.1848021,0.3515817,-6.214208,5.158405e-10,-2.8738896,-1.4957145
living_standard2:1,0.3376925,0.2413341,1.399274,1.617309e-01,-0.1353137,0.8106987
⋮,⋮,⋮,⋮,⋮,⋮,⋮
living_standard3:2,1.4696760,0.3743736,3.925693,8.648029e-05,0.7359172,2.203435
living_standard4:1,0.6264558,0.2141089,2.925874,3.434899e-03,0.2068100,1.046102
living_standard4:2,1.9894933,0.3640658,5.464653,4.638139e-08,1.2759374,2.703049


In [31]:
tibble(summary_multinomial_model6[4:8, 1:2], exp.estimate = round(exp(summary_multinomial_model6[4:8, 2]), 2))


term,estimate,exp.estimate
<chr>,<dbl>,<dbl>
living_standard2:2,0.8238255,2.28
living_standard3:1,0.5344572,1.71
living_standard3:2,1.4696760,4.35
living_standard4:1,0.6264558,1.87
living_standard4:2,1.9894933,7.31


### Multinomial Logistic regression - multiple regression

Let us try a more complex model  with additive interaction terms including multiple regressors. For ease of understanding, we shall skip using the full model with all the regressors. 

In [26]:
multinomial_model_full <- vglm(contraceptive_method ~ living_standard + age + education + media_exposure + children + work,
                           family = multinomial, 
                           data = df)

summary_multinomial_full <- tidy.vglm(multinomial_model_full,
  conf.int = TRUE, conf.level = 0.95
) 
summary_multinomial_full[3:8,]
summary_multinomial_full[9:10,]
summary_multinomial_full[11:16,]
summary_multinomial_full[17:22,]

,term,estimate,std.error,statistic,p.value,conf.low,conf.high
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,living_standard2:1,0.3977553,0.2560965,1.553146,0.120388163,-0.10418455,0.8996952
4,living_standard2:2,0.5723235,0.4227399,1.353843,0.175786406,-0.25623142,1.4008784
5,living_standard3:1,0.4968886,0.2419165,2.053968,0.039978832,0.02274101,0.9710362
6,living_standard3:2,0.9442217,0.3950509,2.390127,0.016842549,0.16993626,1.7185072
7,living_standard4:1,0.7283807,0.2428568,2.999219,0.002706723,0.25239022,1.2043713
8,living_standard4:2,1.2392430,0.3939765,3.145474,0.001658178,0.46706320,2.0114227


,term,estimate,std.error,statistic,p.value,conf.low,conf.high
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
9,age:1,-0.10802866,0.01106169,-9.766014,1.575275e-22,-0.12970919,-0.08634814
10,age:2,-0.03994496,0.01178862,-3.388435,7.029261e-04,-0.06305022,-0.01683970


,term,estimate,std.error,statistic,p.value,conf.low,conf.high
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
11,education2:1,0.1699925,0.2525518,0.6730994,5.008840e-01,-0.32499994,0.6649849
12,education2:2,0.8579722,0.4198818,2.0433659,4.101623e-02,0.03501893,1.6809255
13,education3:1,0.4704315,0.2523657,1.8640862,6.230960e-02,-0.02419627,0.9650592
14,education3:2,1.6799719,0.4071314,4.1263627,3.685458e-05,0.88200899,2.4779348
15,education4:1,0.9764648,0.2580915,3.7834055,1.546971e-04,0.47061474,1.4823148
16,education4:2,2.6919243,0.4069113,6.6155068,3.702809e-11,1.89439286,3.4894557


,term,estimate,std.error,statistic,p.value,conf.low,conf.high
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
17,media_exposure1:1,-0.55759987,0.27923384,-1.9968922,4.583689e-02,-1.1048881,-0.01031161
18,media_exposure1:2,-0.38159730,0.39623823,-0.9630502,3.355223e-01,-1.1582100,0.39501535
19,children:1,0.34508436,0.03804563,9.0702766,1.187152e-19,0.2705163,0.41965242
20,children:2,0.33203471,0.04199712,7.9061307,2.655124e-15,0.2497219,0.41434754
21,work1:1,0.16779980,0.15020549,1.1171349,2.639367e-01,-0.1265976,0.46219716
22,work1:2,0.02231038,0.16756276,0.1331464,8.940776e-01,-0.3061066,0.35072737


The multiple regression shows that the there are significant associations between `contraceptive_method` and 
- high living standard
- age
- high level of education
- number of children